# Entrenando y eligiendo modelo de ML para predecir precios de casas

In [1]:
# Librerías básicas
import pandas as pd
import numpy as np

# Librerías de visualización
import matplotlib.pyplot as plt
import seaborn as sns